In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; float:center}</style>")

In [2]:
from __future__ import division, print_function

# from sys import argv
# import os
# import glob
# import math
import re
import pandas as pd
# import numpy as np
# import scipy as sp
# import scipy.stats as sps
# import matplotlib.pyplot as plt
# import seaborn as sns
# import simplejson as jsn
import requests 
from time import sleep
import json
import re
from urllib import urlretrieve, urlopen
from urllib2 import HTTPError
from bs4 import BeautifulSoup
import datetime
import time

# %matplotlib inline

# sns.set_context('notebook')
# sns.set_style('ticks')

## general

In [3]:
def OpenPage(url):
    try: 
        html = urlopen(url)
    except HTTPError as e: 
        return None

    try:
        pageData = BeautifulSoup(html, 'lxml') #,"html.parser")  #
    except AttributeError as e: 
        return None
    return pageData


def TryFind(search):
    try: 
        result = search
    except AttributeError as e: 
        return None
    if result == None: 
        return None
    return result


# def TryFind(search):
#     try: 
#         result = search
#     except AttributeError as e: 
#         return None
#     if result == None: 
#         return None
#     return result


def GetText(search):
    if TryFind(search) == None:
        return None
    else: 
        try: 
            text = TryFind(search).get_text().strip()
        except AttributeError as e: 
            return None
        if text == None: 
            return None
    return text
    
    
def GetAttribute(search, attribute):
    if TryFind(search) == None:
        return None
    else:
        try: 
            attr = TryFind(search).attrs[attribute]
        except AttributeError as e:
            return None
        if attr == None: 
            return None
        
    return attr


def GetKeys(search):
    try: 
        results = TryFind(search).attrs
    except AttributeError as e:
        return None
    
    if results == None: 
        return '0000-00-00t00:00:00'
    
    return results.keys()


def ScrapeSitemap(url):
    '''returns urls under <loc> tag in sitemap, as text string'''
    sitemap = OpenPage(url)
    taglist = sitemap.findAll('loc')
    urllist = [elem.text for elem in taglist]
    return urllist


def URLFilter(urllist, expression):
    '''filters url list via regex to eliminate non-recipe pages (category aggregations, misc posts, etc)'''
    matches = []
    for url in urllist: 
        match = re.match(expression, url)
        if match: 
            matches += [match.group(0)]
    return matches

## SK-specific

In [4]:
def GetEntryDescription_SK(soup):
    '''get the stuff thats an appendage to the actual recipe body (blog-y part)'''
    
    recipe_text = []
    for paragraph in TryFind(soup.select('div.smittenkitchen-print-hide > p')):
        expression = '.*(\s)*.*[Aa]go:.*'
        if paragraph.a:
            text = paragraph.get_text().strip()
            match = re.match(expression, text)
            if match: 
                continue
            else: recipe_text += [GetText(paragraph)]
        
        else: recipe_text += [GetText(paragraph)]
    return ''.join(recipe_text)


def GetRecipe_old(soup):
    '''get directions & ingredients separately'''
    
    intro, ingredients, directions, meta = [], [], [], []
    
    for item in TryFind(soup.select('div.entry-content > p')):
        if item.br:
            ingredients += [GetText(item)]
        else: 
            directions += [GetText(item)]
    recipe_servings = 'NA'
    recipe_time = 'NA'
    intro = 'NA'

    return intro, ingredients, ''.join(directions), recipe_servings, recipe_time 


def GetRecipe_new(soup):
    '''get directions & ingredients separately'''
    intro = []
    for item in TryFind(soup.findAll('div',{'class':'jetpack-recipe-notes'})):
        if item.a: 
            intro += GetText(item)
        else: intro += GetText(item)
    intro = ''.join(intro)
    
    ingredients = []    
    for item in TryFind(soup.findAll('li',{'class':'jetpack-recipe-ingredient'})):
        ingredients += [GetText(item)]
#         if item.a: 
#             ingredients += [GetText(item)]
#         else: ingredients += [GetText(item)]

            
    directions = []
    for item in TryFind(soup.find('div',{'class':'jetpack-recipe-directions'}).findAll('p')):
        if item == None: 
            directions += 'none'
#         elif item.a: 
#             directions += [GetText(item)]
        else: 
            directions += [GetText(item)]

    recipe_servings = GetText(TryFind(soup.find('li',{'class':'jetpack-recipe-servings'})))
    recipe_time = GetText(TryFind(soup.find('li',{'class':'jetpack-recipe-time'})))
            
    return ''.join(intro), ingredients, ''.join(directions), recipe_servings, recipe_time



def GetRecipeData_SK(soup):
    '''SK underwent a site redesign; find the right way to scrape the recipe data'''
    
    if TryFind(soup.find('div',{'class':'hrecipe jetpack-recipe'})) == None: 
#         print('old')
        intro, ingredients, directions, recipe_servings, recipe_time = GetRecipe_old(soup)

    elif (TryFind(soup.find('div',{'class':'hrecipe jetpack-recipe'})) != None): 
        if (TryFind(soup.find('div',{'class':'hrecipe jetpack-recipe-notes'})) != None) & (TryFind(soup.find('div',{'class':'hrecipe jetpack-recipe-ingredient'})) != None) & (TryFind(soup.find('div',{'class':'hrecipe jetpack-recipe-directions'})) != None): 
#         print('new')
            intro, ingredients, directions, recipe_servings, recipe_time = GetRecipe_new(soup)
        
        else: 
            intro, ingredients, directions, recipe_servings, recipe_time = GetRecipe_old(soup)
    else: 
        intro, ingredients, directions, recipe_servings, recipe_time = GetRecipe_old(soup)

    return intro, ingredients, directions, recipe_servings, recipe_time

In [5]:
def GetCommentData_SK(comment, ID, children, child_id):
    '''get the data from each comment - '''
    comment_info = {}

    name = TryFind(comment.find('b',{'class':'fn author-name'}))
    if TryFind(name.find('a')) == None:
        comment_info['username'] = GetText(TryFind(comment.find('b',{'class':'fn author-name'})))
        comment_info['usersite'] = 'none'

    else: 
        comment_info['username'] = TryFind(comment.find('a')).contents[0]
        comment_info['usersite'] = TryFind(comment.find('a')).attrs['href']

        
    comment_info['usercomment'] = GetText(TryFind(comment.find('div',{'class':'comment-content'}).find('p')))
    comment_info['children'] = children
    comment_info['child_id'] = child_id
    comment_info['commentID'] = ID
    
    comment_info['title'] = TryFind(TryFind(soup.find('h1',{'class':'entry-title'})).find('a').contents[0])    
    comment_info['url'] = GetAttribute(TryFind(soup.find('h1',{'class':'entry-title'})).find('a'), 'href')
    comment_info['recipenumber'] = recipenumber
    
    timestamp_raw = GetText(soup.find('span',{'class':'date'})) +', '+ GetText(soup.find('span',{'class':'time'}))
    comment_info['comment_time'] = datetime.datetime.strptime(timestamp_raw, '%B %d, %Y, %I:%M %p')
                            # datetime.datetime.strptime(time, '%Y-%m-%dt%H:%M:%S').strftime()

    return comment_info



In [6]:
def GetComments_SK(soup, comments_total):
    '''get comments and their place in the hierarchy'''

    comments_data = {}
    comment_has_children, child_id = 'no', 0 

    allcomments = TryFind(soup.find('ol',{'class':'comment-list'}))
    number_of_comments = len(soup.findAll('article',{'class':'comment-body'}))

    comment = TryFind(allcomments.find('article',{'class':'comment-body'}))

    for counter, _ in enumerate(range(0, number_of_comments)):
        comment_ID = comments_total + counter
        nextComment = comment.findNext('article',{'class':'comment-body'})

        if counter < number_of_comments-1:
            if nextComment.parent.parent['class'][0] == 'children':
                comment_has_children, child_id = 'yes', comment_ID+1
            else: comment_has_children, child_id = 'no', 0 
        else: comment_has_children, child_id = 'no', 0 
            
        comments_data[comment_ID] = GetCommentData_SK(comment, comment_ID, comment_has_children, child_id)

        comment = comment.findNext('article')

    return comments_data, number_of_comments

In [7]:
def GetRecipe_SK(soup, ID):
    '''get all the post data from a given url'''
    recipe = {}
    recipe_data = {}
    
    recipe_data['title'] = TryFind(TryFind(soup.find('h1',{'class':'entry-title'})).find('a').contents[0])    
    recipe_data['url'] = GetAttribute(TryFind(soup.find('h1',{'class':'entry-title'})).find('a'), 'href')
    recipe_data['article_id'] = TryFind(TryFind(soup.find('article')).attrs['id'])
    recipe_data['published'] = TryFind(soup.find('time')).attrs['datetime']
    recipe_data['updated'] = TryFind(TryFind(soup.find('time',{'class':'updated'})).attrs['datetime'])
    recipe_data['author'] = TryFind(TryFind(soup.find('span',{'class':'author vcard'})).find('a').contents[0])
    recipe_data['author_url'] = GetAttribute(TryFind(soup.find('span',{'class':'author vcard'})).find('a'), 'href')
    recipe_data['numberofcomments'] = len(TryFind(soup.findAll('article',{'class':'comment-body'})))

    
    recipe_data['introductory_text'] = GetEntryDescription_SK(soup)
    
    recipe_intro, recipe_ingredients, recipe_directions, recipe_servings, recipe_time = GetRecipeData_SK(soup)
    
    recipe_data['recipe_notes'] = recipe_intro
    recipe_data['ingredients'] = recipe_ingredients
    recipe_data['directions'] = recipe_directions
    recipe_data['servings'] = recipe_servings
    recipe_data['time'] = recipe_time
    
    recipe_data['firstimageURL'] = GetAttribute(soup.find('img'), 'src')
    
    recipe[ID] = recipe_data
    
    return recipe, recipe_data['title']


## set preliminaries

## get URLs

In [8]:
url = 'https://smittenkitchen.com'
sk_sitemap = 'https://smittenkitchen.com/sitemap.xml'

# for SK, recipes are listed in <url>/yr/mo/<name> format, other aggregations/lists do not have this same formatting 
sk_regex = 'https://smittenkitchen.com/[0-9]{3,}.*'


urls_sk = URLFilter(ScrapeSitemap(sk_sitemap), sk_regex)

In [9]:
len(urls_sk), urls_sk[0]

(984, u'https://smittenkitchen.com/2016/09/magic-apple-plum-cobbler/')

# Scraper

original scraper, directly adding into pandas df

scraper adding into dict instead, possibly faster?

In [ ]:
urls_sk['']

In [22]:
temp = urls_sk[479:]
temp

[u'https://smittenkitchen.com/2006/12/puff-and-shimmer/',
 u'https://smittenkitchen.com/2006/12/magnificence-au-vin/',
 u'https://smittenkitchen.com/2006/12/utterly-true/',
 u'https://smittenkitchen.com/2007/01/pizza-and-the-limits-of-diy/',
 u'https://smittenkitchen.com/2007/01/superlatively-souper/',
 u'https://smittenkitchen.com/2007/01/in-which-world-peace-eludes-me/',
 u'https://smittenkitchen.com/2007/01/four-breakfasts/',
 u'https://smittenkitchen.com/2007/01/artichoke-gaga/',
 u'https://smittenkitchen.com/2007/01/take-my-wife/',
 u'https://smittenkitchen.com/2007/01/i-blame-babies-for-this/',
 u'https://smittenkitchen.com/2007/01/quiche-me-deadly/',
 u'https://smittenkitchen.com/2007/01/baklasagne/',
 u'https://smittenkitchen.com/2007/01/cake-paradisi/',
 u'https://smittenkitchen.com/2007/01/hibernation-fare/',
 u'https://smittenkitchen.com/2007/01/a-tatin-auditioned/',
 u'https://smittenkitchen.com/2007/01/with-glee-and-ebullience/',
 u'https://smittenkitchen.com/2007/01/the-w

In [ ]:
# temp = urls_sk[:5]
# comments_all, recipes_all = {}, {}
# comments_total = 0

for recipenumber, url in enumerate(temp): 
    print(len(temp)-recipenumber)
    soup = OpenPage(url)
    number = recipenumber + 478
    
    comments, commentnumber = GetComments_SK(soup, comments_total)
    recipe, title = GetRecipe_SK(soup, number)
#     recipe, title = GetRecipe_SK(soup, recipenumber)
    
    recipes_all.update(recipe)
    comments_all.update(comments)
    
    comments_total += commentnumber


#     CommentsFrame = pd.concat([CommentsFrame, CommentsFrame_temp], ignore_index=True)
#     RecipeFrame = pd.concat([RecipeFrame, RecipeFrame_temp], ignore_index=True)   

    time.sleep(2)

    
comments_smittenkitchen = pd.DataFrame.from_dict(comments_all).T
recipes_smittenkitchen = pd.DataFrame.from_dict(recipes_all).T

comments_smittenkitchen.to_csv('comments_smittenkitchen_479.csv', sep=',', encoding='utf-8')
recipes_smittenkitchen.to_csv('recipes_smittenkitchen_479.csv', sep=',', encoding='utf-8')

505
504
503
502
501
500
499
498
497
496
495
494
493
492
491
490
489
488
487
486
485
484
483
482
481
480
479
478
477
476
475
474
473
472
471
470
469
468
467
466
465
464
463
462
461
460
459
458
457
456
455
454
453
452
451
450
449
448
447
446
445
444
443
442
441
440
439
438
437
436
435
434
433
432
431
430
429
428
427
426
425
424
423
422
421
420
419
418
417
416
415
414
413
412
411
410
409
408
407
406
405
404
403
402
401
400
399
398
397
396
395
394
393
392
391
390
389
388
387
386
385
384
383
382
381
380
379
378
377
376
375
374
373
372
371
370
369
368
367
366
365
364
363
362
361
360
359
358
357
356
355
354
353
352
351
350
349
348
347
346
345
344
343
342
341
340
339
338
337
336
335
334
333
332
331
330
329
328
327
326
325
324
323
322
321
320
319
318
317
316
315
314
313
312
311
310
309
308
307
306
305
304
303
302
301
300
299
298
297
296
295
294
293
292
291
290
289
288
287
286
285
284
283
282
281
280
279
278
277
276
275
274
273
272
271
270
269
268
267
266
265
264
263
262
261
260
259
258
257
256


### testing

In [11]:
comments_smittenkitchen = pd.DataFrame.from_dict(comments_all).T
recipes_smittenkitchen = pd.DataFrame.from_dict(recipes_all).T

comments_smittenkitchen.to_csv('comments_smittenkitchen_479.csv', sep=',', encoding='utf-8')
recipes_smittenkitchen.to_csv('recipes_smittenkitchen_479.csv', sep=',', encoding='utf-8')

In [20]:
recipes_smittenkitchen[recipes_smittenkitchen.title =='hazelnut truffles'].url

477    https://smittenkitchen.com/2006/12/mounds-of-a...
Name: url, dtype: object

In [23]:
recipes_smittenkitchen

,article_id,author,author_url,directions,firstimageURL,ingredients,introductory_text,numberofcomments,published,recipe_notes,servings,time,title,updated,url
0,post-29361,deb,https://smittenkitchen.com/author/debsk/,,https://smittenkitchendotcom.files.wordpress.c...,[],"September is my favorite in food, weather and ...",9,2016-09-16T12:07:20+00:00,NA,NA,NA,magic apple plum cobbler,2016-09-16T12:31:12+00:00,https://smittenkitchen.com/2016/09/magic-apple...
1,post-29320,deb,https://smittenkitchen.com/author/debsk/,,https://smittenkitchendotcom.files.wordpress.c...,[],I had lunch with Julia Turshen a couple months...,46,2016-09-13T12:14:13+00:00,NA,NA,NA,homemade merguez with herby yogurt,2016-09-16T12:07:43+00:00,https://smittenkitchen.com/2016/09/homemade-me...
2,post-5183,deb,https://smittenkitchen.com/author/debsk/,Whipped egg whites give this pie a frothy text...,https://smittenkitchendotcom.files.wordpress.c...,[Sweet Potato Buttermilk Pie\nAdapted only sli...,"As many of you have figured out, I’ve got a me...",259,2009-11-17T11:37:41+00:00,NA,NA,NA,sweet potato buttermilk pie,2016-09-15T11:31:59+00:00,https://smittenkitchen.com/2009/11/sweet-potat...
3,post-5631,deb,https://smittenkitchen.com/author/debsk/,,https://smittenkitchendotcom.files.wordpress.c...,[],"People who really, really love chocolate are d...",994,2010-01-30T10:12:13+00:00,NA,NA,NA,best cocoa brownies,2016-09-14T15:48:31+00:00,https://smittenkitchen.com/2010/01/best-cocoa-...
4,post-200,deb,https://smittenkitchen.com/author/debsk/,"Every time, and really, it’s never often enoug...",https://smittenkitchendotcom.files.wordpress.c...,"[, ]",,47,2007-03-07T23:00:24+00:00,NA,NA,NA,sav’h,2016-09-14T14:41:31+00:00,https://smittenkitchen.com/2007/03/savh/
5,post-6035,deb,https://smittenkitchen.com/author/debsk/,Serves 4 to 6Make the dressing: Combine the ca...,https://smittenkitchendotcom.files.wordpress.c...,"[Classic Cobb Salad\nAdapted, barely, from Sav...",When I am considering recipes I might share wi...,217,2010-04-14T22:13:01+00:00,NA,NA,NA,classic cobb salad,2016-09-14T11:24:50+00:00,https://smittenkitchen.com/2010/04/classic-cob...
6,post-7904,deb,https://smittenkitchen.com/author/debsk/,This dish gets an amazing amount of flavor out...,https://smittenkitchendotcom.files.wordpress.c...,"[Cumin Seed Roasted Cauliflower with Yogurt, M...",I do this dreary thing every October where I d...,222,2011-10-20T14:48:04+00:00,NA,NA,NA,cumin seed roasted cauliflower with yogurt,2016-09-14T11:21:06+00:00,https://smittenkitchen.com/2011/10/cumin-seed-...
7,post-29265,deb,https://smittenkitchen.com/author/debsk/,"Look, no one is ever going to marry me for my ...",https://smittenkitchendotcom.files.wordpress.c...,"[, , , One year ago: Caponata\nTwo years ago: ...",,113,2016-09-09T12:01:24+00:00,NA,NA,NA,chocolate pavlova,2016-09-13T12:14:39+00:00,https://smittenkitchen.com/2016/09/chocolate-p...
8,post-10803,deb,https://smittenkitchen.com/author/debsk/,Fudgy Chocolate Sheet CakeThis is the sheet ca...,https://smittenkitchendotcom.files.wordpress.c...,"[Cake layers\n1 cup (2 sticks, 8 ounces or 230...",My son’s first birthday cake was a banana cake...,327,2013-09-20T08:55:10+00:00,NA,NA,NA,fudgy chocolate sheet cake,2016-09-12T17:16:10+00:00,https://smittenkitchen.com/2013/09/fudgy-choco...
9,post-261,deb,https://smittenkitchen.com/author/debsk/,Israeli SaladYou can either toss all of the ve...,https://smittenkitchendotcom.files.wordpress.c...,"[2 medium roma tomatoes, cubed\n1 1-pound Engl...","First I talked about madeleines, and although ...",77,2007-07-03T11:53:33+00:00,NA,NA,NA,israeli salad + pita chips,2016-09-12T17:12:45+00:00,https://smittenkitchen.com/2007/07/a-salad-pal...
